In [1]:
# !pip install google_play_scraper
# !pip install nltk
# !pip install gibberish_detector
# !pip install contractions
# !pip install stanza
# !pip install symspellpy
# !pip install googletrans

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/Sentiment Score

/content/drive/MyDrive/Sentiment Score


In [4]:
import pandas as pd
import re
import itertools
import time
from google_play_scraper import app
from google_play_scraper import Sort, reviews_all
import stanza
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import contractions
from gibberish_detector import detector
import pkg_resources
from symspellpy import SymSpell
from googletrans import Translator
from tqdm import tqdm
import nltk
nltk.download('vader_lexicon')

languages_all = [('aa', 'Afar'), ('ab', 'Abkhazian'), ('af', 'Afrikaans'), ('ak', 'Akan'), ('sq', 'Albanian'), ('am', 'Amharic'), ('ar', 'Arabic'), ('an', 'Aragonese'), ('hy', 'Armenian'), ('as', 'Assamese'), ('av', 'Avaric'), ('ae', 'Avestan'), ('ay', 'Aymara'), ('az', 'Azerbaijani'), ('ba', 'Bashkir'), ('bm', 'Bambara'), ('eu', 'Basque'), ('be', 'Belarusian'), ('bn', 'Bengali'), ('bh', 'Bihari languages'), ('bi', 'Bislama'), ('bo', 'Tibetan'), ('bs', 'Bosnian'), ('br', 'Breton'), ('bg', 'Bulgarian'), ('my', 'Burmese'), ('ca', 'Catalan; Valencian'), ('cs', 'Czech'), ('ch', 'Chamorro'), ('ce', 'Chechen'), ('zh', 'Chinese'), ('cu', 'Church Slavic; Old Slavonic; Church Slavonic; Old Bulgarian; Old Church Slavonic'), ('cv', 'Chuvash'), ('kw', 'Cornish'), ('co', 'Corsican'), ('cr', 'Cree'), ('cy', 'Welsh'), ('cs', 'Czech'), ('da', 'Danish'), ('de', 'German'), ('dv', 'Divehi; Dhivehi; Maldivian'), ('nl', 'Dutch; Flemish'), ('dz', 'Dzongkha'), ('el', 'Greek, Modern (1453-)'), ('en', 'English'), ('eo', 'Esperanto'), ('et', 'Estonian'), ('eu', 'Basque'), ('ee', 'Ewe'), ('fo', 'Faroese'), ('fa', 'Persian'), ('fj', 'Fijian'), ('fi', 'Finnish'), ('fr', 'French'), ('fy', 'Western Frisian'), ('ff', 'Fulah'), ('Ga', 'Georgian'), ('de', 'German'), ('gd', 'Gaelic; Scottish Gaelic'), ('ga', 'Irish'), ('gl', 'Galician'), ('gv', 'Manx'), ('el', 'Greek, Modern (1453-)'), ('gn', 'Guarani'), ('gu', 'Gujarati'), ('ht', 'Haitian; Haitian Creole'), ('ha', 'Hausa'), ('he', 'Hebrew'), ('hz', 'Herero'), ('hi', 'Hindi'), ('ho', 'Hiri Motu'), ('hr', 'Croatian'), ('hu', 'Hungarian'), ('hy', 'Armenian'), ('ig', 'Igbo'), ('is', 'Icelandic'), ('io', 'Ido'), ('ii', 'Sichuan Yi; Nuosu'), ('iu', 'Inuktitut'), ('ie', 'Interlingue; Occidental'), ('ia', 'Interlingua (International Auxiliary Language Association)'), ('id', 'Indonesian'), ('ik', 'Inupiaq'), ('is', 'Icelandic'), ('it', 'Italian'), ('jv', 'Javanese'), ('ja', 'Japanese'), ('kl', 'Kalaallisut; Greenlandic'), ('kn', 'Kannada'), ('ks', 'Kashmiri'), ('ka', 'Georgian'), ('kr', 'Kanuri'), ('kk', 'Kazakh'), ('km', 'Central Khmer'), ('ki', 'Kikuyu; Gikuyu'), ('rw', 'Kinyarwanda'), ('ky', 'Kirghiz; Kyrgyz'), ('kv', 'Komi'), ('kg', 'Kongo'), ('ko', 'Korean'), ('kj', 'Kuanyama; Kwanyama'), ('ku', 'Kurdish'), ('lo', 'Lao'), ('la', 'Latin'), ('lv', 'Latvian'), ('li', 'Limburgan; Limburger; Limburgish'), ('ln', 'Lingala'), ('lt', 'Lithuanian'), ('lb', 'Luxembourgish; Letzeburgesch'), ('lu', 'Luba-Katanga'), ('lg', 'Ganda'), ('mk', 'Macedonian'), ('mh', 'Marshallese'), ('ml', 'Malayalam'), ('mi', 'Maori'), ('mr', 'Marathi'), ('ms', 'Malay'), ('Mi', 'Micmac'), ('mk', 'Macedonian'), ('mg', 'Malagasy'), ('mt', 'Maltese'), ('mn', 'Mongolian'), ('mi', 'Maori'), ('ms', 'Malay'), ('my', 'Burmese'), ('na', 'Nauru'), ('nv', 'Navajo; Navaho'), ('nr', 'Ndebele, South; South Ndebele'), ('nd', 'Ndebele, North; North Ndebele'), ('ng', 'Ndonga'), ('ne', 'Nepali'), ('nl', 'Dutch; Flemish'), ('nn', 'Norwegian Nynorsk; Nynorsk, Norwegian'), ('nb', 'Bokmål, Norwegian; Norwegian Bokmål'), ('no', 'Norwegian'), ('oc', 'Occitan (post 1500)'), ('oj', 'Ojibwa'), ('or', 'Oriya'), ('om', 'Oromo'), ('os', 'Ossetian; Ossetic'), ('pa', 'Panjabi; Punjabi'), ('fa', 'Persian'), ('pi', 'Pali'), ('pl', 'Polish'), ('pt', 'Portuguese'), ('ps', 'Pushto; Pashto'), ('qu', 'Quechua'), ('rm', 'Romansh'), ('ro', 'Romanian; Moldavian; Moldovan'), ('ro', 'Romanian; Moldavian; Moldovan'), ('rn', 'Rundi'), ('ru', 'Russian'), ('sg', 'Sango'), ('sa', 'Sanskrit'), ('si', 'Sinhala; Sinhalese'), ('sk', 'Slovak'), ('sk', 'Slovak'), ('sl', 'Slovenian'), ('se', 'Northern Sami'), ('sm', 'Samoan'), ('sn', 'Shona'), ('sd', 'Sindhi'), ('so', 'Somali'), ('st', 'Sotho, Southern'), ('es', 'Spanish; Castilian'), ('sq', 'Albanian'), ('sc', 'Sardinian'), ('sr', 'Serbian'), ('ss', 'Swati'), ('su', 'Sundanese'), ('sw', 'Swahili'), ('sv', 'Swedish'), ('ty', 'Tahitian'), ('ta', 'Tamil'), ('tt', 'Tatar'), ('te', 'Telugu'), ('tg', 'Tajik'), ('tl', 'Tagalog'), ('th', 'Thai'), ('bo', 'Tibetan'), ('ti', 'Tigrinya'), ('to', 'Tonga (Tonga Islands)'), ('tn', 'Tswana'), ('ts', 'Tsonga'), ('tk', 'Turkmen'), ('tr', 'Turkish'), ('tw', 'Twi'), ('ug', 'Uighur; Uyghur'), ('uk', 'Ukrainian'), ('ur', 'Urdu'), ('uz', 'Uzbek'), ('ve', 'Venda'), ('vi', 'Vietnamese'), ('vo', 'Volapük'), ('cy', 'Welsh'), ('wa', 'Walloon'), ('wo', 'Wolof'), ('xh', 'Xhosa'), ('yi', 'Yiddish'), ('yo', 'Yoruba'), ('za', 'Zhuang; Chuang'), ('zh', 'Chinese'), ('zu', 'Zulu')]
negative_words = {' ad ', ' ads ', 'uninstall', 'advertisement', 'advertising', 'advertise ', 'issue', ' slow', 'irrelevant', ' hang', 'bogus', 'trash', ' crash', 'shit', 'faltu', '3rd class', 'third class', 'bekar', 'third grade', '3rd grade', 'ghatiya', 'pathetic', 'worst'}
positive_words = {'marvellous', 'great', 'mind blowing', 'extraordinary', 'incredible', 'tremendous', 'amazing', 'super', 'splendid', 'mind boggling', 'fantastic', 'wonderful'}
corrections = {
    'aap': 'app',
    'use full': 'useful',
    'wonder full': 'wonderful',
    'exc lent': 'excellent',
    'helped': 'help',
    'out standing': 'outstanding',
    'pls': 'please',
    'plz': 'please',
    'nyc': 'nice',
    'nis': 'nice' 
}
vocab = {'ad', 'aap', 'nyc', 'nis', 'plz', 'pls', 'fix'}

stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment')

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt"
)
bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt"
)
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)


Detector = detector.create_from_model('gibberish-detector.model')

translator = Translator()


all_reviews = None
scored_reviews = None

def get_review_func(package_name, lang = 'en'):
    try:
        result = reviews_all(package_name, sleep_milliseconds=0, lang=lang)
        return result
    except:
        return None

def monthyear_func(x):
    return str(x.year) + "{:02d}".format(x.month) 

def get_reviews(package_name, languages):
    combined_df = None
    print(f'Scraping {package_name}...')
    st = time.time()
    for language in tqdm(languages):
        review_dict = get_review_func(package_name, language[0])
        try:
            df = pd.DataFrame(review_dict)
            if type(combined_df) == type(None):
                combined_df = df.copy(deep=True)
            else:
                combined_df = combined_df.append(df.copy(deep=True), ignore_index = True)
        except:
            pass
    print(f'Scraping done in {round(time.time() - st,2)}s')
    combined_df = combined_df.drop_duplicates()
    return combined_df.rename(columns = {'score': 'rating'})

def translate(df):
    print('Translating...')
    cnt = 0
    en_content = []
    for review in tqdm(df['content']):
        try:
            en_content.append(translator.translate(review).text)
        except:
            cnt += 1
            en_content.append(review)
    df['en_content'] = en_content
    print(f"cnt: {cnt}")
    return df

def max_seq(sent, n=2):
    sent = re.sub(' +', ' ', sent)
    clean_word_list = []
    for text in sent.split(' '):
        result = []
        for k, g in itertools.groupby(text):
            result.extend(list(g)[:n])
        result = ''.join(result)
        clean_word_list.append(result)
    return ' '.join(clean_word_list)

def clean_text(df):
    corpus = []
    for txt in tqdm(df['en_content']):
        try:
            txt = txt.lower()
            #fix contractions
            txt = max_seq(txt)
            txt = contractions.fix(txt, slang=True)
            txt = re.sub(r'[^a-zA-Z ]',' ',txt)
            txt = sym_spell.word_segmentation(txt, max_edit_distance = 2)[0]
            txt = txt.split(' ')
            wordlist = []
            for word in txt:
                if word in vocab:
                    wordlist.append(word)
                else:
                    correction = sym_spell.lookup_compound(word, max_edit_distance=2)
                    if not Detector.is_gibberish(correction[0].term): 
                        wordlist.append(correction[0].term)
            txt = ' '.join(wordlist)
            txt = ' ' + txt + ' '
            for misspelled_word in corrections.keys():
                if misspelled_word in txt:
                    txt = txt.replace(misspelled_word, corrections[misspelled_word])
        except:
            pass
        corpus.append(txt)  
    df['cleaned_text'] = corpus
    print('cleaning complete')
    return df

class Vader():
    def __init__(self):
        self.vader = SentimentIntensityAnalyzer()
    
    def score(self, text):
        #Vader Sentiment Intensity Analyzer 
        return self.vader.polarity_scores(text)['compound']
    
    def predict(self, df):
        print('applying score')
        df['polarity score'] = df['cleaned_text'].apply(self.score)
        stanza_scores = []
        for i in tqdm(range(len(df))):
            score = df.iloc[i]['polarity score']
            if score==0:
                doc = nlp(df.iloc[i]['cleaned_text'])
                for sentence in doc.sentences:
                    score += 0.5*(sentence.sentiment-1)
                if len(doc.sentences):
                    score /= len(doc.sentences)
            stanza_scores.append(score)
        df['stanza_scores'] = stanza_scores
        stanza_and_rule = []
        for i in tqdm(range(len(df))):
            score = df.iloc[i]['stanza_scores']
            if score==0:
                rating,cnt = 0,0
                for word in positive_words:
                    if word in df.iloc[i]['cleaned_text']:
                        rating += 0.5
                        cnt += 1
                for word in negative_words:
                    if word in df.iloc[i]['cleaned_text']:
                        rating -= 0.5
                        cnt += 1
                if cnt:
                    score = rating/cnt
            stanza_and_rule.append(score)
        df['stanza_and_rule'] = stanza_and_rule
        df['sr_combined'] = 5*abs(df['rating']-3)*df['stanza_and_rule']
        return df

def rating_distribution_bias(df):
    total = len(df)
    low_rating_cnt = len(df[(df['rating']==1)|(df['rating']==2)])
    high_rating_cnt = len(df[(df['rating']==4)|(df['rating']==5)])
    low_perc = (low_rating_cnt/total)*100
    high_perc = (high_rating_cnt/total)*100
    if high_perc>75:
        return 2
    elif high_perc>50:
        return 1
    elif low_perc>75:
        return -2
    elif low_perc>50:
        return -1
    return 0

def negative_scaling(df, column_name):
    df[column_name + '_scaled'] = df[column_name]
    df.loc[df['rating']==1, column_name + '_scaled'] = df[column_name]*2
    df.loc[df['rating']==2, column_name + '_scaled'] = df[column_name]*1.5
    return df

def get_score(df):
    global scored_reviews
    clf = Vader()
    df = clf.predict(df)
    bias = rating_distribution_bias(df)
    df = df[(df['rating']-3)*df['stanza_and_rule']>=0] #remove contradictions
    df = negative_scaling(df, 'sr_combined') #for negative scaling
    scored_reviews = df.copy(deep=True)
    score = df[df['sr_combined_scaled']!=0]['sr_combined_scaled'].mean() #for negative scaling
    return score + bias

def reviews_score(package, threshold = 50000):
    global all_reviews
    reviews_cnt = app(package)['ratings']
    if reviews_cnt>threshold:
        df = get_reviews(package, languages=[('en', 'English')]) 
        df['en_content'] = list(df['content'])   
    else:
        df = get_reviews(package, languages=languages_all)
        df = translate(df)
    df = clean_text(df)
    all_reviews = df.copy(deep=True)
    score = get_score(df)
    return score

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


INFO:stanza:Downloading default packages for language: en (English)...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| sentiment | sstplus  |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: sentiment
INFO:stanza:Done loading processors!


In [5]:
reviews_score('com.cama.app.huge80sclock')

Scraping com.cama.app.huge80sclock...


100%|██████████| 203/203 [05:46<00:00,  1.71s/it]


Scraping done in 346.86s
Translating...


100%|██████████| 20837/20837 [14:35<00:00, 23.80it/s]


cnt: 0


100%|██████████| 20837/20837 [35:14<00:00,  9.85it/s]


cleaning complete
applying score


100%|██████████| 20837/20837 [00:20<00:00, 1004.53it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


7.194430068769983

In [10]:
print(len(all_reviews), len(scored_reviews))

20837 19851


In [11]:
scored_reviews = scored_reviews[['reviewId','sr_combined_scaled']]
df_merged = all_reviews.set_index('reviewId').join(scored_reviews.set_index('reviewId'), how = 'outer')
df_merged.reset_index(inplace=True)
df_merged['sr_combined_scaled'] = df_merged['sr_combined_scaled'].fillna(0)
df_merged.sort_values(by = 'at', inplace=True)
df_merged.drop(['userName', 'userImage'], axis=1, inplace=True)
df_merged.drop_duplicates(inplace=True)
print(len(df_merged))
df_merged.head()

20837


,reviewId,content,rating,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,en_content,cleaned_text,sr_combined_scaled
18242,e0a68a14-5434-41a9-9bf7-c0d9ddebb34a,Cool,5,0,1.1,2018-08-25 19:23:16,None,NaT,Cool,cool,3.182
8645,6978712c-2eba-4d8e-a2d1-fa88918b538d,Togliete la status bar del cellulare in alto....,2,0,1.2,2018-09-23 12:31:36,"Ciao. Ho migliorato l'app, ora è più personali...",2019-05-31 11:27:06,Remove the cell phone status bar at the top ....,remove the cell phone status bar at the top,0.000
14798,b5ae1ff1-e7f8-4f6e-92f6-3fed68c63140,Bella app,5,1,None,2018-10-04 12:55:42,;),2018-10-04 12:56:52,Beautiful app,beautiful app,5.994
11434,8c545564-0f31-4917-b8c4-20734a6e7fa5,Molto bello e carino,5,1,1.3.3,2018-10-10 18:28:10,Grazie! :),2018-10-11 08:11:10,Very nice and cute,very nice and cute,7.474
17179,d3015bbd-fa05-4d67-b2bd-0fc725b0289e,"App accettabile, sarebbe gradito che il piccol...",3,0,1.3.3,2018-10-12 18:16:06,Mi chiedi una cosa difficilissima! D'altronde ...,2018-10-19 14:59:49,"Acceptable app, it would be appreciated if the...",acceptable app it would be appreciated if the...,0.000


In [12]:
clf = Vader()
df_merged = clf.predict(df_merged)
df_merged.drop(['polarity score', 'stanza_scores', 'replyContent', 'repliedAt', 'cleaned_text', 'sr_combined'], axis=1, inplace=True)
df_merged.rename(columns = {'stanza_and_rule': 'polarity score'}, inplace=True)
df_merged.head()

applying score


100%|██████████| 20837/20837 [00:20<00:00, 1002.94it/s]


,reviewId,content,rating,thumbsUpCount,reviewCreatedVersion,at,en_content,sr_combined_scaled,polarity score
18242,e0a68a14-5434-41a9-9bf7-c0d9ddebb34a,Cool,5,0,1.1,2018-08-25 19:23:16,Cool,3.182,0.3182
8645,6978712c-2eba-4d8e-a2d1-fa88918b538d,Togliete la status bar del cellulare in alto....,2,0,1.2,2018-09-23 12:31:36,Remove the cell phone status bar at the top ....,0.000,0.2023
14798,b5ae1ff1-e7f8-4f6e-92f6-3fed68c63140,Bella app,5,1,None,2018-10-04 12:55:42,Beautiful app,5.994,0.5994
11434,8c545564-0f31-4917-b8c4-20734a6e7fa5,Molto bello e carino,5,1,1.3.3,2018-10-10 18:28:10,Very nice and cute,7.474,0.7474
17179,d3015bbd-fa05-4d67-b2bd-0fc725b0289e,"App accettabile, sarebbe gradito che il piccol...",3,0,1.3.3,2018-10-12 18:16:06,"Acceptable app, it would be appreciated if the...",0.000,0.8442


In [15]:
# df_merged.to_csv('/content/drive/MyDrive/Sentiment Score/com.cama.app.huge80sclock.csv', index=False)